In [75]:
%reload_ext sql

In [76]:
%sql sqlite:///C:\Users\User\Desktop\sql\chinook.db

In [77]:
%%sql
SELECT *
FROM sqlite_master

 * sqlite:///C:\Users\User\Desktop\sql\chinook.db
Done.


type,name,tbl_name,rootpage,sql
table,albums,albums,2,"CREATE TABLE ""albums"" ( [AlbumId] INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, [Title] NVARCHAR(160) NOT NULL, [ArtistId] INTEGER NOT NULL, FOREIGN KEY ([ArtistId]) REFERENCES ""artists"" ([ArtistId]) ON DELETE NO ACTION ON UPDATE NO ACTION )"
table,sqlite_sequence,sqlite_sequence,3,"CREATE TABLE sqlite_sequence(name,seq)"
table,artists,artists,4,"CREATE TABLE ""artists"" ( [ArtistId] INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, [Name] NVARCHAR(120) )"
table,customers,customers,5,"CREATE TABLE ""customers"" ( [CustomerId] INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, [FirstName] NVARCHAR(40) NOT NULL, [LastName] NVARCHAR(20) NOT NULL, [Company] NVARCHAR(80), [Address] NVARCHAR(70), [City] NVARCHAR(40), [State] NVARCHAR(40), [Country] NVARCHAR(40), [PostalCode] NVARCHAR(10), [Phone] NVARCHAR(24), [Fax] NVARCHAR(24), [Email] NVARCHAR(60) NOT NULL, [SupportRepId] INTEGER, FOREIGN KEY ([SupportRepId]) REFERENCES ""employees"" ([EmployeeId]) ON DELETE NO ACTION ON UPDATE NO ACTION )"
table,employees,employees,8,"CREATE TABLE ""employees"" ( [EmployeeId] INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, [LastName] NVARCHAR(20) NOT NULL, [FirstName] NVARCHAR(20) NOT NULL, [Title] NVARCHAR(30), [ReportsTo] INTEGER, [BirthDate] DATETIME, [HireDate] DATETIME, [Address] NVARCHAR(70), [City] NVARCHAR(40), [State] NVARCHAR(40), [Country] NVARCHAR(40), [PostalCode] NVARCHAR(10), [Phone] NVARCHAR(24), [Fax] NVARCHAR(24), [Email] NVARCHAR(60), FOREIGN KEY ([ReportsTo]) REFERENCES ""employees"" ([EmployeeId]) ON DELETE NO ACTION ON UPDATE NO ACTION )"
table,genres,genres,10,"CREATE TABLE ""genres"" ( [GenreId] INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, [Name] NVARCHAR(120) )"
table,invoices,invoices,11,"CREATE TABLE ""invoices"" ( [InvoiceId] INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, [CustomerId] INTEGER NOT NULL, [InvoiceDate] DATETIME NOT NULL, [BillingAddress] NVARCHAR(70), [BillingCity] NVARCHAR(40), [BillingState] NVARCHAR(40), [BillingCountry] NVARCHAR(40), [BillingPostalCode] NVARCHAR(10), [Total] NUMERIC(10,2) NOT NULL, FOREIGN KEY ([CustomerId]) REFERENCES ""customers"" ([CustomerId]) ON DELETE NO ACTION ON UPDATE NO ACTION )"
table,invoice_items,invoice_items,13,"CREATE TABLE ""invoice_items"" ( [InvoiceLineId] INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, [InvoiceId] INTEGER NOT NULL, [TrackId] INTEGER NOT NULL, [UnitPrice] NUMERIC(10,2) NOT NULL, [Quantity] INTEGER NOT NULL, FOREIGN KEY ([InvoiceId]) REFERENCES ""invoices"" ([InvoiceId]) ON DELETE NO ACTION ON UPDATE NO ACTION, FOREIGN KEY ([TrackId]) REFERENCES ""tracks"" ([TrackId]) ON DELETE NO ACTION ON UPDATE NO ACTION )"
table,media_types,media_types,15,"CREATE TABLE ""media_types"" ( [MediaTypeId] INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, [Name] NVARCHAR(120) )"
table,playlists,playlists,16,"CREATE TABLE ""playlists"" ( [PlaylistId] INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL, [Name] NVARCHAR(120) )"


In [78]:
%%sql
--  1. Create a trigger to log when a new invoice is inserted
CREATE TABLE IF NOT EXISTS InvoiceLog(
    LogId INTEGER PRIMARY KEY AUTOINCREMENT,
    InvoiceId INTEGER,
    LogMessage TEXT,
    LogDate DATETIME DEFAULT CURRENT_TIMESTAMP
)
;
CREATE TRIGGER IF NOT EXISTS LogNewInvoice
AFTER INSERT ON invoices
BEGIN 
    INSERT INTO InvoiceLog(InvoiceId,LogMessage)
    VALUES(NEW.InvoiceId, "New invoice Added");
END;

 * sqlite:///C:\Users\User\Desktop\sql\chinook.db
Done.
Done.


[]

In [40]:
%%sql
-- to visualize te tables for eazy access
SELECT name
FROM sqlite_master
WHERE type = "table"


 * sqlite:///C:\Users\User\Desktop\sql\chinook.db
Done.


name
albums
sqlite_sequence
artists
customers
employees
genres
invoices
invoice_items
media_types
playlists


In [52]:
%%sql
-- 1. Use CASE to categorize customers based on total spending
SELECT 
    FirstName || " " || LastName as Customer_Name,
    sum(i.Total) as Total_spent,
    CASE 
        WHEN  sum(i.Total) > 45 THEN  "Platinum"
        WHEN  sum(i.Total) BETWEEN 40 and 45 THEN  "Gold"
        ELSE "silver"
    END as Customer_Tier 
FROM customers c
JOIN
    invoices i
ON
    c.CustomerId = i.CustomerId
GROUP BY c.CustomerId
ORDER BY Total_spent DESC

LIMIT 5

 * sqlite:///C:\Users\User\Desktop\sql\chinook.db
Done.


Customer_Name,Total_spent,Customer_Tier
Helena Holý,49.62,Platinum
Richard Cunningham,47.62,Platinum
Luis Rojas,46.62,Platinum
Ladislav Kovács,45.62,Platinum
Hugh O'Reilly,45.62,Platinum


In [65]:
%%sql
--   2. Combine a CTE + CASE to find the top genre per country with a label
with GenreRevenue as ( 
    SELECT 
        g.Name as Genre,
        c.Country country,
        sum(it.UnitPrice * it.Quantity) as Revenue
    FROM 
        genres g
    JOIN 
        tracks t
    ON
        g.GenreId = t.GenreId	
    JOIN
        invoice_items it
    ON 
        t.TrackId = it.TrackId
    JOIN
        invoices i
    ON
        it.InvoiceId = i.InvoiceId
    JOIN
         customers c
    ON
       i.CustomerId = c.CustomerId
    GROUP BY c.country, g.Name
)
SELECT 
    Genre,
    Country,
    Revenue,
    CASE
        WHEN Revenue = (
            SELECT 
                MAX(Revenue)
            FROM 
                GenreRevenue g2
            WHERE g1.Country = g1.Country
        ) THEN "Top Genre"
        ELSE "None"
                            
    END as Label

FROM 
    GenreRevenue g1
ORDER BY Country, Revenue DESC  
LIMIT 5

 * sqlite:///C:\Users\User\Desktop\sql\chinook.db
Done.


Genre,country,Revenue,Label
Alternative & Punk,Argentina,8.91,None
Rock,Argentina,8.91,None
Latin,Argentina,7.92,None
Metal,Argentina,6.93,None
Easy Listening,Argentina,1.98,None


In [70]:
%%sql
--   4. Challenge: Genre sales growth year by year
with GenreSales as (
    SELECT 
        g.Name as Genre,
        round(sum(it.UnitPrice * it.Quantity),3) as Revenue,
        strftime("%Y", i.InvoiceDate) as Year
    FROM 
        genres g
    JOIN 
        tracks t
    ON
        g.GenreId = t.GenreId	
    JOIN
        invoice_items it
    ON 
        t.TrackId = it.TrackId
    JOIN
        invoices i
    ON
        it.InvoiceId = i.InvoiceId
    GROUP BY Year, Genre
)

SELECT *
FROM GenreSales
LIMIT 5

 * sqlite:///C:\Users\User\Desktop\sql\chinook.db
Done.


Genre,Revenue,Year
Alternative & Punk,62.37,2009
Blues,10.89,2009
Bossa Nova,0.99,2009
Easy Listening,2.97,2009
Electronica/Dance,1.98,2009
